<a href="https://colab.research.google.com/github/ummehanikonok16/ImageNet/blob/inceptionV3/inceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = r'/content/drive/MyDrive/AAV/train_val_photos-20240207T143515Z-001/train_val_photos'

In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

In [ ]:
from torchvision.datasets import ImageFolder
dataset = ImageFolder(root=data_dir, transform=transform)

In [ ]:
from torch.utils.data import DataLoader, random_split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

img_width, img_height = 299, 299
batch_size = 32
epochs = 10

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))



In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_294 (Conv2D)         (None, 149, 149, 32)         864       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_282 (B  (None, 149, 149, 32)         96        ['conv2d_294[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_282 (Activation  (None, 149, 149, 32)         0         ['batch_normalization_28

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

Epoch 1/10
48/48 [==============================] - 1127s 21s/step - loss: 0.7055 - accuracy: 0.7609 - val_loss: 1861.6221 - val_accuracy: 0.2005
Epoch 2/10
48/48 [==============================] - 921s 19s/step - loss: 0.3375 - accuracy: 0.8682 - val_loss: 19.5895 - val_accuracy: 0.4870
Epoch 3/10
48/48 [==============================] - 942s 20s/step - loss: 0.2603 - accuracy: 0.8979 - val_loss: 21.0481 - val_accuracy: 0.4896
Epoch 4/10
48/48 [==============================] - 919s 19s/step - loss: 0.2447 - accuracy: 0.9170 - val_loss: 9.4122 - val_accuracy: 0.4844
Epoch 5/10
48/48 [==============================] - 952s 20s/step - loss: 0.1777 - accuracy: 0.9328 - val_loss: 7.6202 - val_accuracy: 0.4974
Epoch 6/10
48/48 [==============================] - 951s 20s/step - loss: 0.1829 - accuracy: 0.9394 - val_loss: 20.4261 - val_accuracy: 0.2188
Epoch 7/10
48/48 [==============================] - 945s 20s/step - loss: 0.1250 - accuracy: 0.9499 - val_loss: 23.6724 - val_accuracy: 0.487

In [ ]:
# Evaluate the model on the validation set
eval_result = model.evaluate(val_generator, steps=val_generator.samples // batch_size)

# Print the evaluation result
print("Validation Loss: {:.4f}".format(eval_result[0]))
print("Validation Accuracy: {:.2f}%".format(eval_result[1] * 100))



12/12 [==============================] - 62s 5s/step - loss: 3.6697 - accuracy: 0.6745
Validation Loss: 3.6697
Validation Accuracy: 67.45%
